# Linear Programming

Linear programming is a mathematical technique to achieve the best outcome (such as maximum profit, or least possible cost), whilst having various constraints.

For our course, we will be using the `pulp` library which you can find more information __[here](https://www.coin-or.org/PuLP/pulp.html)__.

## Problem 4

A publisher has orders for 600 copies of a certain text from San Francisco and 400 copies from Sacramento. 

The company has 700 copies in a warehouse in Novato and 800 copies in a warehouse in Lodi. 

It costs $5 to ship a text from Novato to San Francisco, but it costs $10 to ship it to Sacramento. It costs $15 to ship a text from Lodi to San Francisco, but it costs $4 to ship it from Lodi to Sacramento.

How many copies should the company ship from each warehouse to San Francisco and Sacramento to fill the order at the least cost?

<br/>

## Setting up the Problem

The first task is to set up the problem by:
- Creating an `LpProblem`
- Defining the variables (including any bounds, and data type)

In this case we have four variables that are obtained from the possible combinations of shipments from each warehouse to each store. Their data type must be `Integer` since we cannot have partials.

In [18]:
# import libraries
from pulp import *

# create the linear programming model
lp = LpProblem("Publisher", LpMinimize)

# define variables
a = LpVariable(name='NV_SF', lowBound=0, cat='Integer')
b = LpVariable(name='NV_SC', lowBound=0, cat='Integer')
c = LpVariable(name='LD_SF', lowBound=0, cat='Integer')
d = LpVariable(name='LD_SC', lowBound=0, cat='Integer')

## Determine the Objective

Next step is to determine the objective

In this case we have: 
- Novato to San Francisco (a) costs $5
- Novato to Sacramento (b) costs $10
- Lodi to San Francisco (c) costs $15
- Lodi to Sacramento (d) costs $4

We want to minimize the delivery cost, so we have $min(p) = 5a + 10b + 15c + 4d $

This can be coded as following

In [19]:
lp.objective = (5*a) + (10*b) + (15*c) + (4*d)

## Setting up the Constraints

The next step is to plot out the constraints. In this case we have the following:
- Delivery from Novato cannot exceed 700 copies
- Delivery from Lodi cannot exceed 800 copies
- San Francisco store must have exactly 600 copies
- Sacramento store must have exactly 400 copies

If we represent this in a table we have:

| Novato to SF (a) | Novato to Sac. (b) | Lodi to SF (c) | Lodi to Sac. (d) | Limit |
| --- | --- | --- | --- | --- |
| 1 | 1 | 0 | 0 | <=700 |
| 0 | 0 | 1 | 1 | <=800 |
| 1 | 0 | 1 | 0 | = 600 |
| 0 | 1 | 0 | 1 | = 400 |



In [20]:
lp.addConstraint(a+b <= 700, 'NV supply limit')
lp.addConstraint(c+d <= 800, 'LD supply limit')
lp.addConstraint(a+c == 600, 'SF Demand')
lp.addConstraint(b+d == 400, 'SC demand')

## Solving the Problem

The last step is to solve the problem. This can be done by calling the `solve` function.

In [21]:
# solve the linear programming problem
# 1 = Optimal, 2 = Infeasible, 3 = Unbounded, 4 = Undefined, 5 = Not Solved

status = lp.solve(PULP_CBC_CMD(msg=0))
print(status)

# print solution
for var in lp.variables():
    print(var,'=',value(var))
    
print('OPT',value(lp.objective))

1
LD_SC = 400.0
LD_SF = 0.0
NV_SC = 0.0
NV_SF = 600.0
OPT 4600.0


If we look at the outcome we have:

status = 1 - The staus is optimal (it can be not solved, infeasible, unbounded or undef)

Deliveries should be done From Lodi to Sacramento (400), and Novato to San Francisco (600).
This will cost 4600.